# Logistic Regression

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from tqdm.auto import tqdm

In [3]:
df_collision = pd.read_csv("./data/collisions_final.csv")
df_collision.head(2)

,police_force,number_of_vehicles,day_of_week,time,first_road_class,road_type,speed_limit,light_conditions,weather_conditions,road_surface_conditions,is_severe,month,day_of_year,is_trunk,is_near_pedestrian_crossing,is_urban,has_special_conditions_at_site,is_carriageway_hazard,is_near_junction
0,metropolitan_police,1,sunday,01:00,c,one_way_street,20,darkness___lights_lit,other_adverse_weather_condition,wet_or_damp,0,january,1,0,1,1,0,0,1
1,metropolitan_police,3,sunday,02:00,unclassified,single_carriageway,30,darkness___lights_lit,fine_no_high_winds,dry,0,january,1,0,1,1,0,0,1


## Prepare data

In [4]:
df_full_train, df_test = train_test_split(df_collision, test_size=0.2, random_state=11)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=11)

df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = df_train["is_severe"].values
y_val = df_val["is_severe"].values
y_test = df_test["is_severe"].values

del df_train["is_severe"]
del df_val["is_severe"]
del df_test["is_severe"]

In [5]:
df_train.head()

,police_force,number_of_vehicles,day_of_week,time,first_road_class,road_type,speed_limit,light_conditions,weather_conditions,road_surface_conditions,month,day_of_year,is_trunk,is_near_pedestrian_crossing,is_urban,has_special_conditions_at_site,is_carriageway_hazard,is_near_junction
0,metropolitan_police,2,friday,20:00,a,dual_carriageway,30,darkness___lights_lit,fine_no_high_winds,dry,september,258,0,1,1,0,0,1
1,metropolitan_police,2,saturday,06:00,a,dual_carriageway,20,darkness___lights_lit,fine_no_high_winds,dry,april,112,0,1,1,0,0,1
2,police_scotland,2,saturday,17:00,unclassified,single_carriageway,30,daylight,fine_no_high_winds,dry,june,154,0,0,1,0,0,0
3,police_scotland,1,saturday,17:00,a,single_carriageway,60,daylight,fine_no_high_winds,dry,april,112,0,0,0,0,0,0
4,west_yorkshire,2,saturday,00:00,unclassified,single_carriageway,30,darkness___lights_lit,fine_no_high_winds,wet_or_damp,may,126,0,0,1,0,0,0


In [6]:
len(df_train)

41425

In [7]:
df_train.columns

Index(['police_force', 'number_of_vehicles', 'day_of_week', 'time',
       'first_road_class', 'road_type', 'speed_limit', 'light_conditions',
       'weather_conditions', 'road_surface_conditions', 'month', 'day_of_year',
       'is_trunk', 'is_near_pedestrian_crossing', 'is_urban',
       'has_special_conditions_at_site', 'is_carriageway_hazard',
       'is_near_junction'],
      dtype='object')

In [8]:
# We use all the columns in our dataset
x_train_dicts = df_train.to_dict(orient="records")
dv = DictVectorizer(sparse=False).fit(x_train_dicts)
X_train = dv.transform(x_train_dicts)

val_dicts = df_val.to_dict(orient="records")
X_val = dv.transform(val_dicts)

In [9]:
len(dv.get_feature_names_out())

126

In [10]:
len(X_train)

41425

## Train base model and evaluate it with auc_score

**Why am I not using the default `lbfgs` algorithm?**

The default optimization solver in LogisticRegression is `lbfgs` but it `lbfgs` is known to struggle with large datasets (the training dataset is > 40K rows), data with many categorical features (most of my features are categorical) and high dimensional data. Our dataset is not considered to be very high dimensional. The feature to sample ratio is only about 1:200. 

I was curious to play around with the `penalty` parameter (see explanation below) to see how it interacted. The `lbfgs` does not support `l1`.

In [11]:
model = LogisticRegression(solver="liblinear",random_state = 1, penalty="l1")
model.fit(X_train, y_train)

LogisticRegression(penalty='l1', random_state=1, solver='liblinear')

In [12]:
y_pred = model.predict_proba(X_val)[:,1]
roc_auc_score_base = roc_auc_score(y_val, y_pred) 
roc_auc_score_base

np.float64(0.615508821652256)

**Observations**

`roc_auc_score`: 0.6155

Our initial model didn't achieve a great auc_score: 0.6155.

**Looking at intercept**

The intercept is under 0.5, we could interpret this as meaning that the model predicts less than 50% chance that the collision is severe.

In [13]:
def sigmoid(z):
    return 1/(1 + np.exp(-z)) 


w0 = model.intercept_[0]
w0, sigmoid(w0)

(np.float64(-0.1787536656643347), np.float64(0.45543019819837377))

**Looking at model coefficients**

I used the `l1` regularization (`penalty`) parameter which unlike the default `l2` can shrink coefficients to zero effectively removing them. (A give a more thorough description of this below).

I was curious to see which features had the smallest coefficients and the least impact on the target variable.

In another iteration of this project, I could think about removing them.

In [14]:
model_coefficients = []

for feature, coef in zip(dv.get_feature_names_out(), model.coef_[0]):
    model_coefficients.append((feature, coef))

df_model_coeff = pd.DataFrame(model_coefficients, columns=["feature_name","model_coefficient"])
df_model_coeff[(df_model_coeff["model_coefficient"] == 0)] 

,feature_name,model_coefficient
11,first_road_class=c,0.0
22,light_conditions=darkness___lights_unlit,0.0
27,month=december,0.0
33,month=may,0.0
42,police_force=city_of_london,0.0
44,police_force=cumbria,0.0
76,police_force=thames_valley,0.0
77,police_force=warwickshire,0.0
107,time=13:00,0.0
110,time=16:00,0.0


## Model Tuning

I want to tune a couple of parameters in the model: `C` and `penalty`.

I will use the KFold split method to evaluate the model on different subsets of the full training set and then take the average `roc_auc_score`. This will also allow me to look at the standard deviation of each model: if this is large then the model is more unstable.

**penalty parameter: l1 vs l2**

The default `penalty` parameter for LogisticRegression models is `l2`: this is what we used in lectures. I wanted to try tuning the model with `l1` regularization because I was unsure if all the features were really relevant.

The `penalty` parameter determines the type of regularization:

- `l2`: this type of regularization reduces the magnitude of all coefficients. The advice is to use this type of regularization when you believe all features are useful but you want to prevent one feature from having too much influence. This shrinking of coefficients is a way of preventing overfitting
  
- `l1`: in l2 regularization, coefficients are reduced but not to zero. In `l1` regularization coefficients are allowed to shrink to exactly zero. This makes this type of regularization a good choice if you suspect some of your features are irrelevant because it effectively removes them.

**C parameter**

We have tuned the `C` parameter in lectures. It controls the regularization strength. 

Smaller C means stronger regularization and more penalty on coefficients (i.e. coefficients may be smaller or in `l1`'s case more likely to be exactly zero). Larger C means weaker regularization and less penalty on coefficients (i.e. coefficients may be larger).



In [15]:
def train(df_training, y_training, C=1.0, penalty="l2"):
    """Train the dataset

    Parameters
    ---------
    df_training: training dataframe
    y_training: np.array of y training values
    C: regularization strength parameter to pass to LogisticRegression function, must be between (0, inf]
    penalty: regularization type parameter (l1 or l2)
    """
    dicts = df_training.to_dict(orient='records')
    dv = DictVectorizer(sparse=False).fit(dicts)
    X_train = dv.transform(dicts)

    model = LogisticRegression(solver="liblinear", C=C, penalty=penalty)
    model.fit(X_train, y_training)

    return dv, model

In [16]:
def predict(df_val, dv, model):
    """Calculate predictions for given dataset

    Parameters
    ---------
    df_val: the validation dataset to perform the prediction on
    dv: DictVectorizer to use to transform the validation dataset
    model: The trained model to use to calculate the predictions
    """
    dicts = df_val.to_dict(orient='records')
    X_val = dv.transform(dicts) 

    y_pred = model.predict_proba(X_val)[:,1]
    return y_pred

In [17]:
def model_tuning(df, kfold_n_splits=10, reg_strength_list=[1.0], reg_type_list=['l2']):
    """Evaluate the model with different model parameters

    Parameters
    ---------
    df: The dataset to use to extract training and validation datasets and perform evaluation
    kfold_n_slits: the number of groups the df should be split into 
    reg_strength_list: list of the C (regularization strength) parameters to evaluate
    reg_type_list: list of the penalty (regularization type) parameters to evaluate

    """
    kfold = KFold(n_splits=kfold_n_splits, shuffle=True, random_state=1)
    
    totals = []

    for reg_type in tqdm(reg_type_list, total=len(reg_type_list)):
        for reg_strength in tqdm(reg_strength_list,total=len(reg_strength_list)):
            scores = []

            for train_idx, val_idx in kfold.split(df):
                df_t = df.iloc[train_idx]
                df_v = df.iloc[val_idx]
            
                y_t = df_t["is_severe"].values
                y_v = df_v["is_severe"].values
            
                del df_t["is_severe"]
                del df_v["is_severe"]
                dv, model =train(df_t, y_t, C=reg_strength, penalty=reg_type)
                y_pred = predict(df_v, dv, model)
                auc = roc_auc_score(y_v, y_pred)
                scores.append(auc)
                
            totals.append((reg_type,reg_strength, np.round(np.mean(scores),3), np.round(np.std(scores),3)))

    return totals

In [18]:
# Warning! This took just under 6 minutes on my machine
kfold_n_splits=10
reg_strength_list = [0.00001, 0.001, 0.01, 0.1, 0.5, 1.0, 5.0, 10.0]
reg_type_list=['l1','l2']
totals = model_tuning(df_full_train, kfold_n_splits, reg_strength_list, reg_type_list)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

In [20]:
df_tuning = pd.DataFrame(totals, columns=["reg_type","reg_strength","avg_roc", "avg_std"])

df_tuning.sort_values(by="avg_roc", ascending=False)

,reg_type,reg_strength,avg_roc,avg_std
3,l1,0.10000,0.615,0.008
4,l1,0.50000,0.615,0.008
11,l2,0.10000,0.615,0.008
5,l1,1.00000,0.614,0.008
7,l1,10.00000,0.614,0.008
13,l2,1.00000,0.614,0.008
10,l2,0.01000,0.614,0.008
6,l1,5.00000,0.614,0.008
14,l2,5.00000,0.614,0.008
15,l2,10.00000,0.614,0.008


### Model Tuning Conclusions

After training the model with different parameters, I found that the largest average `roc_auc_score` I could get was no better than my base model i.e. 0.615. Not particularly high, given that we were seeing models achieving 0.8 in lectures!

This top score of 0.615 happens when the `penalty=l1,C=0.1`.

The type of regularization didn't seem to matter too much: The next best model parameters produce an avg_roc of 0.616. Both `l2` and `l1` models produce this score suggesting that the choice of regularization is perhaps not too important.

By contrast, average `roc_auc_score` was consistently less when `C`, the regularization strength, was under 0.01. We know that smaller C means more of a penalty on coefficients. For our `l1` algorithm, this means that more of the features are likely to be set to zero. This finding makes me think that there are not as many irrelevant features as I feared.

In [21]:
roc_auc_score_tuning = 0.615

## Final model

I will use the full training set to train a model with the parameters found in the model tuning evaluation above. 

In [22]:
# Prepare datasets: `is_severe` still exists in df_full_train so we need to remove it and extract the y_full_train

df_full_train = df_full_train.reset_index(drop=True)

y_full_train = df_full_train["is_severe"].values

del df_full_train["is_severe"]


dicts_full_train = df_full_train.to_dict(orient="records")

# Set parameters
penalty_final = "l1"
C_final=0.1

# Train model and get model and DictVectorizer
dv, model =train(df_full_train, y_full_train, C=C_final, penalty=penalty_final)

y_pred_test = predict(df_test, dv, model)
roc_auc_score_final = roc_auc_score(y_test, y_pred_test)
roc_auc_score_final

np.float64(0.6143927539297247)

In [23]:
roc_auc_score_tuning - np.round(roc_auc_score_final, 3)

np.float64(0.0010000000000000009)

### Final model evaluation

Our final model has a slightly lower score (0.614) than the model we built with the training set alone: it is less by around 0.001, this is a fraction of a percent so we can say that our model didn't overfit and generalised quite well to unseen data.

That being said, the `roc_auc_score` for this model is quite low. 